# Import & Path

## Import

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Rescaling
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import os
import math
import pickle

np.random.seed(42)

## Path

In [ ]:
basePath = os.getcwd()
trainPath = os.path.join(os.getcwd(), "Images")
print(trainPath)

# Default Configuration

In [ ]:
height = 128
width = 128
channels = 3
label = "categorical"

# Model & Training Preparation

## Load Dataset

In [ ]:
def load_img(path, batch_size):
  train = tf.keras.preprocessing.image_dataset_from_directory(
    path,
    labels = "inferred",
    label_mode = label,
    color_mode = "rgb",
    batch_size = batch_size,
    image_size = (height, width),
    shuffle = True,
    seed = 42,
    subset = "training",
    validation_split = 0.2
  )

  val = tf.keras.preprocessing.image_dataset_from_directory(
    path,
    labels = "inferred",
    label_mode = label,
    color_mode = "rgb",
    batch_size = batch_size,
    image_size = (height, width),
    shuffle = True,
    seed = 42,
    subset = "validation",
    validation_split = 0.2
  )
  
  return train, val

## Train Model

In [ ]:
def TrainModel(train, val, learning_rate, epochs, pretrain_model, name, model_dir):
    model = Sequential()
    model.add(Rescaling(1./255))
    model.add(pretrain_model)
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(43, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=math.pow(10, (learning_rate * -1))), metrics=['accuracy'])
    history = model.fit(train, epochs=epochs, validation_data=val)
    model.save(os.path.join(model_dir, (name + '.h5')))

    return history

## Evaluate Model

In [ ]:
def EvaluateModel(name, history, epochs, model_dir):
    acc = history.history['accuracy']
    loss = history.history['loss']
    
    if 'val_accuracy' in history.history:
        val_acc = history.history['val_accuracy']
        val_loss = history.history['val_loss']

    epochs_range = range(epochs)

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    
    if 'val_accuracy' in history.history:
        plt.plot(epochs_range, val_acc, label='Validation Accuracy')

    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')

    if 'val_accuracy' in history.history:
        plt.plot(epochs_range, val_loss, label='Validation Loss')

    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.savefig(os.path.join(model_dir, (name + '.png')))
    plt.show()

## Create Pretrain Model

In [ ]:
pretrain_vgg16 = tf.keras.applications.vgg16.VGG16(
    include_top = False,
    input_shape = (height, width, channels),
    pooling = 'avg',
    classes = 43,
    weights = 'imagenet'
)

pretrain_vgg16.trainable = False

pretrain_resnet50 = tf.keras.applications.ResNet50(
    include_top = False,
    input_shape = (height, width, channels),
    pooling = 'avg',
    classes = 43,
    weights = 'imagenet'
)

pretrain_resnet50.trainable = False

pretrain_inceptionv3 = tf.keras.applications.inception_v3.InceptionV3(
    include_top = False,
    input_shape = (height, width, channels),
    pooling = 'avg',
    classes = 43,
    weights = 'imagenet'
)

pretrain_inceptionv3.trainable = False

pretrain_xception = tf.keras.applications.xception.Xception(
    include_top = False,
    input_shape = (height, width, channels),
    pooling = 'avg',
    classes = 43,
    weights = 'imagenet'
)

pretrain_xception.trainable = False

## Training Function

In [ ]:
def TrainProcess(train, val, batch_size, learning_rate, epochs, pretrain_model, model_name, enhance_type):
    name = '{}-{}-{}-{}-{}'.format(model_name, batch_size, learning_rate, epochs, enhance_type)
    model_dir = os.path.join(basePath, '{}-model'.format(enhance_type))
    history = TrainModel(train, val, learning_rate, epochs, pretrain_model, name, model_dir)

    with open(os.path.join(model_dir, (name+ '.pickle')) , 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

    EvaluateModel(name, history, epochs, model_dir)

In [ ]:
def Train(train, val, batch_size, learning_rate, epochs, enhance_type="none"):
    TrainProcess(train, val, batch_size, learning_rate, epochs, pretrain_vgg16, 'vgg16', enhance_type)
    TrainProcess(train, val, batch_size, learning_rate, epochs, pretrain_resnet50, 'resnet50', enhance_type)
    TrainProcess(train, val, batch_size, learning_rate, epochs, pretrain_inceptionv3, 'inceptionv3', enhance_type)
    TrainProcess(train, val, batch_size, learning_rate, epochs, pretrain_xception, 'xception', enhance_type)

# Train Models

## Default Configuration

In [ ]:
batch_size = 32
learning_rate = 3
epochs = 50

## No Image Enhancement

In [ ]:
train, val = load_img(trainPath, batch_size)
Train(train, val, batch_size, learning_rate, epochs)

## With Image Enhancement

In [ ]:
enhance = "gaussian"
train, val = load_img(os.path.join(basePath, (enhance + "-train")), batch_size)
Train(train, val, batch_size, learning_rate, epochs, enhance)

In [ ]:
enhance = "sharpen"
train, val = load_img(os.path.join(basePath, (enhance + "-train")), batch_size)
Train(train, val, batch_size, learning_rate, epochs, enhance)

In [ ]:
enhance = "clahe"
train, val = load_img(os.path.join(basePath, (enhance + "-train")), batch_size)
Train(train, val, batch_size, learning_rate, epochs, enhance)

In [ ]:
enhance = "combo"
train, val = load_img(os.path.join(basePath, (enhance + "-train")), batch_size)
Train(train, val, batch_size, learning_rate, epochs, enhance)

## Robust Model

In [ ]:
train = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(basePath, "robust-train"),
    labels = "inferred",
    label_mode = label,
    color_mode = "rgb",
    batch_size = batch_size,
    image_size = (height, width),
    shuffle = True,
    seed = 42,
)

Train(train, None, batch_size, learning_rate, epochs, "robust")

## Manual Tuning Parameter

Default tuning parameter configuration mention below:
| Model    | Batch Size | Learning Rate |
|--------- |------------|---------------|
| Xception | 32         | 0.001         |
| Xception | 64         | 0.0001        |
| Xception | 32         | 0.001         |
| Xception | 64         | 0.0001        |

Note:
- Xception was selected based on the previous best model
- Model with 32 batch size and 0.001 has been trained in the previous stage

In [ ]:
batch_size = 32
learning_rate = 4

In [ ]:
enhance = "gaussian"
train, val = load_img(os.path.join(basePath, (enhance + "-train")), batch_size)
TrainProcess(train, val, batch_size, learning_rate, epochs, pretrain_xception, 'xception', enhance)

In [ ]:
batch_size = 64
learning_rate = 3

In [ ]:
enhance = "gaussian"
train, val = load_img(os.path.join(basePath, (enhance + "-train")), batch_size)
TrainProcess(train, val, batch_size, learning_rate, epochs, pretrain_xception, 'xception', enhance)

In [ ]:
batch_size = 64
learning_rate = 4

In [ ]:
enhance = "gaussian"
train, val = load_img(os.path.join(basePath, (enhance + "-train")), batch_size)
TrainProcess(train, val, batch_size, learning_rate, epochs, pretrain_xception, 'xception', enhance)